# Fine-tuning a model on a token classification task

## Loading the dataset

## Preprocessing the data

In [1]:
from transformers import AutoTokenizer
import os
import optuna
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoConfig
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np
from datasets import load_from_disk, concatenate_datasets
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Inicializando a variável particoes
particoes = {}
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
# atch_size = 8
print(particoes.keys())

label_all_tokens = True
# Carregar o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Caminho correto para as partições
base_path = "../models/"

# Carregar as partições diretamente do caminho especificado
for i in range(1,11):
    partition_path = f"{base_path}dat_division_{i}"
    print(f"Verificando partição: {partition_path}")  # Para depuração
    
    if os.path.exists(partition_path):  # Verifica se o diretório existe
        try:
            particoes[f"particao_{i}"] = load_from_disk(partition_path)
        except Exception as e:
            print(f"Erro ao carregar a partição {partition_path}: {e}")
    else:
        print(f"Partição {partition_path} não encontrada!")  # Alerta se o diretório não existir  # Verificar quais partições foram carregadas corretamente
print("Partições carregadas:", list(particoes.keys()))


# Criar o mapeamento de label2id
label2id = {
    'O': 0,                                      # Fora de qualquer entidade
    'B-CARACTERISTICA_SENSORIAL_AROMA': 1,       # Início da entidade relacionada ao aroma
    'I-CARACTERISTICA_SENSORIAL_AROMA': 2,       # Continuação da entidade relacionada ao aroma
    'B-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': 3,# Início da entidade de consistência
    'I-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': 4,# Continuação da entidade de consistência
    'B-CARACTERISTICA_SENSORIAL_COR': 5,         # Início da entidade de cor
    'I-CARACTERISTICA_SENSORIAL_COR': 6,         # Continuação da entidade de cor
    'B-CARACTERISTICA_SENSORIAL_SABOR': 7,       # Início da entidade de sabor
    'I-CARACTERISTICA_SENSORIAL_SABOR': 8,       # Continuação da entidade de sabor
    'B-CLASSIFICACAO_BEBIDA': 9,                 # Início da classificação da bebida
    'I-CLASSIFICACAO_BEBIDA': 10,                # Continuação da classificação da bebida
    'B-EQUIPAMENTO_DESTILACAO': 11,              # Início da entidade de equipamento de destilação
    'I-EQUIPAMENTO_DESTILACAO': 12,              # Continuação do equipamento de destilação
    'B-GRADUACAO_ALCOOLICA': 13,                 # Início da graduação alcoólica
    'I-GRADUACAO_ALCOOLICA': 14,                 # Continuação da graduação alcoólica
    'B-NOME_BEBIDA': 15,                         # Início do nome da bebida
    'I-NOME_BEBIDA': 16,                         # Continuação do nome da bebida
    'B-NOME_LOCAL': 17,                          # Início do nome de local
    'I-NOME_LOCAL': 18,                          # Continuação do nome de local
    'B-NOME_ORGANIZACAO': 19,                    # Início do nome da organização
    'I-NOME_ORGANIZACAO': 20,                    # Continuação do nome da organização
    'B-NOME_PESSOA': 21,                         # Início do nome de pessoa
    'I-NOME_PESSOA': 22,                         # Continuação do nome de pessoa
    'B-PRECO': 23,                               # Início da entidade de preço
    'I-PRECO': 24,                               # Continuação da entidade de preço
    'B-RECIPIENTE_ARMAZENAMENTO': 25,            # Início da entidade de recipiente de armazenamento
    'I-RECIPIENTE_ARMAZENAMENTO': 26,            # Continuação do recipiente de armazenamento
    'B-TEMPO': 27,                               # Início da entidade de tempo (geral)
    'I-TEMPO': 28,                               # Continuação da entidade de tempo
    'B-TEMPO_ARMAZENAMENTO': 29,                 # Início da entidade de tempo de armazenamento
    'I-TEMPO_ARMAZENAMENTO': 30,                 # Continuação da entidade de tempo de armazenamento
    'B-TIPO_MADEIRA': 31,                        # Início da entidade tipo de madeira
    'I-TIPO_MADEIRA': 32,                        # Continuação da entidade tipo de madeira
    'B-VOLUME': 33,                              # Início da entidade de volume
    'I-VOLUME': 34,                              # Continuação da entidade de volume
}



# Exibir o mapeamento de labels
print(f"Label2ID mapping: {label2id}")

def tokenize_and_align_labels(examples):
    # Verificando se 'sentences' está presente nos dados
    if "sentences" not in examples:
        raise KeyError("A chave 'sentences' não foi encontrada nos dados.")

    tokenized_inputs = []
    labels = []

    for sentence in examples["sentences"]:
        # Tokenizando os tokens de cada sentença
        tokenized_sentence = tokenizer(sentence["tokens"], truncation=True, is_split_into_words=True, max_length=512)

        word_ids = tokenized_sentence.word_ids()  # Pega os ids dos tokens
        sentence_labels = sentence["labels"]  # Labels da sentença

        previous_word_idx = None
        label_ids = []          # Atribuindo as labels corretamente
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token (e.g., [CLS], [SEP])
            elif word_idx != previous_word_idx:
                # Verificando se a label existe em label2id
                label = sentence_labels[word_idx]
                if label not in label2id:
                    print(f"Label {label} não encontrada em label2id!")  # Print de depuração
                label_ids.append(label2id.get(label, -1))  # Usa -1 se a label não for encontrada
            else:
                label_ids.append(label2id.get(sentence_labels[word_idx], -1) if label_all_tokens else -100)

            previous_word_idx = word_idx

        tokenized_inputs.append(tokenized_sentence)
        labels.append(label_ids)

    # Atribuindo as labels ao tokenized_inputs
    for i, tokenized_sentence in enumerate(tokenized_inputs):
        tokenized_sentence["labels"] = labels[i]

    # Retorna os dados tokenizados e alinhados com as labels
    return {"input_ids": [x["input_ids"] for x in tokenized_inputs],
            "attention_mask": [x["attention_mask"] for x in tokenized_inputs],
            "labels": labels}

# Aplicar a tokenização a todas as partições de dados
for i in range(1,11):
    print(f"Tokenizando partição {i}...")
    particoes[f"particao_{i}"] = particoes[f"particao_{i}"].map(tokenize_and_align_labels, batched=True)

print("Tokenização aplicada com sucesso!")


dict_keys([])
Verificando partição: ../models/dat_division_1
Verificando partição: ../models/dat_division_2
Verificando partição: ../models/dat_division_3
Verificando partição: ../models/dat_division_4
Verificando partição: ../models/dat_division_5
Verificando partição: ../models/dat_division_6
Verificando partição: ../models/dat_division_7
Verificando partição: ../models/dat_division_8
Verificando partição: ../models/dat_division_9
Verificando partição: ../models/dat_division_10
Partições carregadas: ['particao_1', 'particao_2', 'particao_3', 'particao_4', 'particao_5', 'particao_6', 'particao_7', 'particao_8', 'particao_9', 'particao_10']
Label2ID mapping: {'O': 0, 'B-CARACTERISTICA_SENSORIAL_AROMA': 1, 'I-CARACTERISTICA_SENSORIAL_AROMA': 2, 'B-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': 3, 'I-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': 4, 'B-CARACTERISTICA_SENSORIAL_COR': 5, 'I-CARACTERISTICA_SENSORIAL_COR': 6, 'B-CARACTERISTICA_SENSORIAL_SABOR': 7, 'I-CARACTERISTICA_SENSORIAL_SABOR': 8, 'B

## Fine-tuning the model

In [2]:
def criar_modelo(num_labels, attn_dropout, hidden_dropout):
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels=num_labels,
        hidden_dropout_prob=hidden_dropout,
        attention_probs_dropout_prob=attn_dropout
    )
    return model

In [3]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

def objective(trial, train_set, val_set, num_labels):
    # Amostragem de hiperparâmetros
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    epochs = trial.suggest_categorical("epochs", [2, 3, 4, 8, 16, 32, 64, 128])
    attn_dropout = trial.suggest_uniform("attention_dropout", 0.0, 0.5)
    hidden_dropout = trial.suggest_uniform("hidden_dropout", 0.0, 0.5)
    weight_decay = trial.suggest_uniform("weight_decay", 0.01, 0.05)

    training_args = TrainingArguments(
        output_dir="./optuna_results",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        logging_dir="./optuna_logs",
        logging_steps=10,
        report_to="none",
    )

    model = criar_modelo(num_labels, attn_dropout, hidden_dropout)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator= data_collator,
        train_dataset=train_set,
        eval_dataset=val_set,
        compute_metrics=compute_metrics
    )

    trainer.train()
    val_metrics = trainer.evaluate()

    return val_metrics["eval_f1_macro"]  # Otimização baseada no F1 macro


In [4]:
import os
import torch
import pandas as pd
import random
import numpy as np
from torch.utils.data import Subset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
import optuna
from optuna.samplers import TPESampler

# Garantir que os diretórios existam
checkpoints_dir = "./checkpoints"
logs_dir = "./logs_final"
final_results_dir = "./final_results"
optuna_db = "sqlite:///optuna_study.db"

os.makedirs(checkpoints_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)
os.makedirs(final_results_dir, exist_ok=True)

torch.cuda.empty_cache()

num_particoes = 10

# Função para obter uma amostra de 0.1% dos dados
def get_subset(dataset, fraction=1):
    total_size = len(dataset)
    subset_size = int(total_size * fraction)
    indices = random.sample(range(total_size), subset_size)
    return Subset(dataset, indices)

# Função para computar as métricas
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    # Considerando apenas as classes com prefixo 'B-'
    for pred, label in zip(predictions, labels):
        true_labels.extend([l for l in label if l != -100 and l % 2 == 1])  # Apenas 'B-...'
        pred_labels.extend([p for p, l in zip(pred, label) if l != -100 and l % 2 == 1])  # Apenas 'B-...'

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average=None, labels = [i for i in range(1, 34, 2)])  # 'B-...'
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(true_labels, pred_labels, average="macro")
    accuracy = accuracy_score(true_labels, pred_labels)

    return {
        "precision_per_class": precision.tolist(),
        "recall_per_class": recall.tolist(),
        "f1_per_class": f1.tolist(),
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "accuracy": accuracy
    }

# Carregar progresso se houver checkpoints
test_results = []
completed_iterations = set()

checkpoint_file = os.path.join(checkpoints_dir, "completed_iterations.txt")
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        completed_iterations = set(map(int, f.read().splitlines()))

num_labels = len(set(label for dataset in particoes.values() for sentence in dataset["sentences"] for label in sentence["labels"]))

import torch
import optuna
import os
import numpy as np
from transformers import TrainingArguments, Trainer
from optuna.samplers import TPESampler






In [5]:


# Definição da partição de validação
validacao_set = particoes["particao_8"]

# Definir conjunto de treino excluindo a partição de validação
train_sets_validacao = [particoes[f"particao_{j}"] for j in range(1,11) if j != 8]
train_set_validacao = torch.utils.data.ConcatDataset(train_sets_validacao)

# Otimização de hiperparâmetros com Optuna na partição 7
study = optuna.create_study(direction="maximize", sampler=TPESampler(), storage=optuna_db, load_if_exists=True)
study.optimize(lambda trial: objective(trial, train_set_validacao, validacao_set, num_labels), n_trials=25)

best_hyperparams = study.best_params
print(f"Melhores hiperparâmetros na partição de validação: {best_hyperparams}")




[I 2025-05-05 18:27:52,235] A new study created in RDB with name: no-name-dc8775b7-5251-49ff-b636-d5b70fe35cb7
/tmp/ipykernel_1706032/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_1706032/5382319.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  attn_dropout = trial.suggest_uniform("attention_dropout", 0.0, 0.5)
/tmp/ipykernel_1706032/5382319.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  hidden_dropout = trial.suggest_uniform("hidden_dropout", 0.0, 0.

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.556300,0.438763,"[0.3867403314917127, 0.0, 1.0, 0.2857142857142857, 0.0, 0.0, 1.0, 0.6342494714587738, 0.9518272425249169, 0.0, 1.0, 0.9368421052631579, 1.0, 1.0, 0.9076923076923077, 0.7332521315468941, 0.9770491803278688]","[0.8045977011494253, 0.0, 0.029411764705882353, 0.046948356807511735, 0.0, 0.0, 0.9955555555555555, 0.9302325581395349, 0.9168, 0.0, 0.28865979381443296, 0.9222797927461139, 0.3624454148471616, 0.7734375, 0.8309859154929577, 0.8326417704011065, 0.9802631578947368]","[0.5223880597014925, 0.0, 0.05714285714285714, 0.08064516129032258, 0.0, 0.0, 0.9977728285077951, 0.754242614707731, 0.9339853300733496, 0.0, 0.448, 0.9295039164490861, 0.532051282051282, 0.8722466960352423, 0.8676470588235294, 0.7797927461139896, 0.9786535303776683]",0.491517,0.396103,0.397912,0.705028
2,0.346700,0.284346,"[0.48859934853420195, 0.0, 0.9, 0.5268817204301075, 0.9787234042553191, 1.0, 1.0, 0.8993993993993994, 0.9850993377483444, 0.926829268292683, 0.9591836734693877, 0.9368421052631579, 0.9629629629629629, 1.0, 0.896774193548387, 0.8969072164948454, 0.994991652754591]","[0.8620689655172413, 0.0, 0.7941176470588235, 0.2300469483568075, 0.233502538071066, 0.44329896907216493, 0.9955555555555555, 0.9286821705426357, 0.952, 0.40860215053763443, 0.9690721649484536, 0.9222797927461139, 0.9082969432314411, 0.8515625, 0.9788732394366197, 0.9626556016597511, 0.9802631578947368]","[0.6237006237006237, 0.0, 0.84375, 0.3202614379084967, 0.3770491803278688, 0.6142857142857143, 0.9977728285077951, 0.9138062547673532, 0.96826688364524, 0.5671641791044776, 0.9641025641025641, 0.9295039164490861, 0.9348314606741573, 0.919831223628692, 0.936026936026936, 0.9286190793862575, 0.9875724937862469]",0.598050,0.517537,0.534439,0.835717
3,0.260000,0.220047,"[0.5742971887550201, 0.9285714285714286, 0.875, 0.7851239669421488, 0.9375, 0.8545454545454545, 1.0, 0.9495268138801262, 0.9948453608247423, 0.9622641509433962, 0.9797979797979798, 0.9368421052631579, 0.9493087557603687, 1.0, 0.896774193548387, 0.9573901464713716, 0.994991652754591]","[0.8218390804597702, 0.40625, 0.9264705882352942, 0.4460093896713615, 0.6852791878172588, 0.9690721649484536, 0.9955555555555555, 0.9333333333333333, 0.9264, 0.8225806451612904, 1.0, 0.9222797927461139, 0.8995633187772926, 0.8515625, 0.9788732394366197, 0.9944674965421854, 0.9802631578947368]","[0.6761229314420804, 0.5652173913043478, 0.9, 0.5688622754491018, 0.7917888563049853, 0.9082125603864735, 0.9977728285077951, 0.9413604378420641, 0.9594034797017399, 0.8869565217391304, 0.9897959183673469, 0.9295039164490861, 0.9237668161434978, 0.919831223628692, 0.936026936026936, 0.9755766621438263, 0.9875724937862469]",0.649032,0.606658,0.619074,0.901170
4,0.194800,0.202724,"[0.4889589905362776, 0.9183673469387755, 0.8873239436619719, 0.8461538461538461, 0.9738562091503268, 0.8245614035087719, 1.0, 0.9494470774091627, 0.9948892674616695, 0.9705882352941176, 0.9797979797979798, 0.9368421052631579, 0.9806763285024155, 1.0, 0.9084967320261438, 0.9768707482993197, 0.994991652754591]","[0.8908045977011494, 0.703125, 0.9264705882352942, 0.15492957746478872, 0.7563451776649747, 0.9690721649484536, 0.9955555555555555, 0.931782945736434, 0.9344, 0.8870967741935484, 1.0, 0.9222797927461139, 0.8864628820960698, 0.875, 0.9788732394366197, 0.9930843706777317, 0.9802631578947368]","[0.6313645621181263, 0.7964601769911505, 0.9064748201438849, 0.2619047619047619, 0.8514285714285714, 0.8909952606635071, 0.9977728285077951, 0.9405320813771518, 0.9636963696369637, 0.9269662921348315, 0.9897959183673469, 0.9295039164490861, 0.9311926605504587, 0.9333333333333333, 0.9423728813559322, 0.9849108367626886, 0.9875724937862469]",0.625273,0.591422,0.594651,0.900737
5,0.190900,0.192704,"[0.6024590163934426, 0.8676470588235294, 0.8904109589041096, 0.9333333333333333, 0.9607843137254902, 0.9038461538461539, 1.0, 0.9539682

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-05-05 18:39:16,774] Trial 0 finished with value: 0.6002027376064873 and parameters: {'learning_rate': 2.306181585057309e-05, 'batch_size': 32, 'epochs': 8, 'attention_dropout': 0.05390781655424054, 'hidden_dropout': 0.48884020979974446, 'weight_decay': 0.02417015784401369}. Best is trial 0 with value: 0.6002027376064873.
/tmp/ipykernel_1706032/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_1706032/5382319.py:10: Futu

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.269800,0.235344,"[0.4225352112676056, 0.9090909090909091, 0.967741935483871, 0.7083333333333334, 0.9379310344827586, 1.0, 0.995575221238938, 0.9313572542901716, 0.9914965986394558, 0.9602649006622517, 0.9791666666666666, 0.9368421052631579, 0.9396551724137931, 1.0, 0.9078947368421053, 0.9846796657381616, 0.994991652754591]","[0.8620689655172413, 0.15625, 0.8823529411764706, 0.07981220657276995, 0.6903553299492385, 0.8969072164948454, 1.0, 0.9255813953488372, 0.9328, 0.7795698924731183, 0.9690721649484536, 0.9222797927461139, 0.9519650655021834, 0.859375, 0.971830985915493, 0.9778699861687413, 0.9802631578947368]","[0.5671077504725898, 0.26666666666666666, 0.9230769230769231, 0.14345991561181434, 0.7953216374269005, 0.9456521739130435, 0.9977827050997783, 0.9284603421461898, 0.9612530915086562, 0.8605341246290801, 0.9740932642487047, 0.9295039164490861, 0.9457700650759219, 0.9243697478991597, 0.9387755102040817, 0.981263011797363, 0.9875724937862469]",0.648648,0.576598,0.586278,0.877980
2,0.244700,0.194318,"[0.4968944099378882, 0.8923076923076924, 0.9591836734693877, 0.8863636363636364, 0.9878787878787879, 0.9306930693069307, 1.0, 0.964573268921095, 0.9948892674616695, 0.95, 0.9797979797979798, 0.9368421052631579, 0.9953271028037384, 1.0, 0.9452054794520548, 0.9703903095558546, 0.994991652754591]","[0.9195402298850575, 0.90625, 0.6911764705882353, 0.18309859154929578, 0.8274111675126904, 0.9690721649484536, 1.0, 0.9286821705426357, 0.9344, 0.9193548387096774, 1.0, 0.9222797927461139, 0.9301310043668122, 0.90625, 0.971830985915493, 0.9972337482710927, 0.9802631578947368]","[0.6451612903225806, 0.8992248062015504, 0.8034188034188035, 0.3035019455252918, 0.9005524861878453, 0.9494949494949495, 1.0, 0.9462875197472354, 0.9636963696369637, 0.9344262295081968, 0.9897959183673469, 0.9295039164490861, 0.9616252821670429, 0.9508196721311475, 0.9583333333333334, 0.9836289222373806, 0.9875724937862469]",0.588346,0.555073,0.559520,0.910056
3,0.141800,0.181444,"[0.5618374558303887, 0.8840579710144928, 0.9833333333333333, 0.9871794871794872, 0.968944099378882, 0.9494949494949495, 1.0, 0.96, 0.9949409780775716, 0.9553072625698324, 0.9797979797979798, 0.9368421052631579, 1.0, 1.0, 0.9452054794520548, 0.97165991902834, 0.998324958123953]","[0.9137931034482759, 0.953125, 0.8676470588235294, 0.3615023474178404, 0.7918781725888325, 0.9690721649484536, 0.9911111111111112, 0.9302325581395349, 0.944, 0.9193548387096774, 1.0, 0.9222797927461139, 0.9606986899563319, 0.9140625, 0.971830985915493, 0.995850622406639, 0.9802631578947368]","[0.6958424507658644, 0.9172932330827067, 0.921875, 0.5292096219931272, 0.8715083798882681, 0.9591836734693877, 0.9955357142857143, 0.9448818897637795, 0.9688013136288999, 0.936986301369863, 0.9897959183673469, 0.9295039164490861, 0.9799554565701559, 0.9551020408163265, 0.9583333333333334, 0.9836065573770492, 0.9892116182572614]",0.554377,0.530576,0.535401,0.922410
4,0.091400,0.173213,"[0.6083650190114068, 0.8732394366197183, 0.9836065573770492, 0.9897959183673469, 0.9512195121951219, 0.9591836734693877, 1.0, 0.9755301794453507, 0.9949494949494949, 0.9615384615384616, 0.9797979797979798, 0.9368421052631579, 1.0, 1.0, 0.9452054794520548, 0.9769335142469471, 0.998324958123953]","[0.9195402298850575, 0.96875, 0.8823529411764706, 0.45539906103286387, 0.7918781725888325, 0.9690721649484536, 0.9911111111111112, 0.9271317829457364, 0.9456, 0.9408602150537635, 1.0, 0.9222797927461139, 0.9606986899563319, 0.90625, 0.971830985915493, 0.995850622406639, 0.9802631578947368]","[0.7322654462242563, 0.9185185185185185, 0.9302325581395349, 0.6237942122186495, 0.8642659279778393, 0.9641025641025641, 0.9955357142857143, 0.9507154213036566, 0.9696472518457753, 0.9510869565217391, 0.9897959183673469, 0.9295039164490861, 0.9799554565701559, 0.9508196721311475, 0.9583333333333334, 0.9863013698630136, 0.989211

...

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-05-06 11:58:03,664] Trial 23 finished with value: 0.5842782401509429 and parameters: {'learning_rate': 1.8961664742218338e-06, 'batch_size': 8, 'epochs': 64, 'attention_dropout': 0.2663599011210435, 'hidden_dropout': 0.0014221826717191188, 'weight_decay': 0.013499968107997178}. Best is trial 14 with value: 0.6583227918224134.
/tmp/ipykernel_1706032/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_1706032/5382319.py:10:

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.190000,0.192200,"[0.5490909090909091, 0.78, 0.875, 0.9130434782608695, 0.975609756097561, 0.9550561797752809, 1.0, 0.9590834697217676, 0.9897260273972602, 0.9204545454545454, 1.0, 0.9368421052631579, 0.9903381642512077, 1.0, 0.92, 0.9768707482993197, 0.994991652754591]","[0.867816091954023, 0.609375, 0.8235294117647058, 0.29577464788732394, 0.8121827411167513, 0.8762886597938144, 1.0, 0.9085271317829458, 0.9248, 0.8709677419354839, 0.9587628865979382, 0.9222797927461139, 0.8951965065502183, 0.8671875, 0.971830985915493, 0.9930843706777317, 0.9802631578947368]","[0.6726057906458798, 0.6842105263157895, 0.8484848484848485, 0.44680851063829785, 0.8864265927977839, 0.9139784946236559, 1.0, 0.9331210191082803, 0.9561621174524401, 0.8950276243093923, 0.9789473684210527, 0.9295039164490861, 0.9403669724770642, 0.9288702928870293, 0.9452054794520548, 0.9849108367626886, 0.9875724937862469]",0.655671,0.607411,0.622175,0.898136
2,0.134500,0.139430,"[0.6977777777777778, 0.8709677419354839, 0.9516129032258065, 0.9777777777777777, 0.9772727272727273, 0.9583333333333334, 1.0, 0.9705882352941176, 0.9949238578680203, 0.9398907103825137, 1.0, 0.9368421052631579, 1.0, 1.0, 0.9452054794520548, 0.9796472184531886, 0.998324958123953]","[0.9022988505747126, 0.84375, 0.8676470588235294, 0.6197183098591549, 0.8730964467005076, 0.9484536082474226, 1.0, 0.9209302325581395, 0.9408, 0.9247311827956989, 0.979381443298969, 0.9222797927461139, 0.9650655021834061, 0.90625, 0.971830985915493, 0.9986168741355463, 0.9802631578947368]","[0.7869674185463659, 0.8571428571428571, 0.9076923076923077, 0.7586206896551724, 0.9222520107238605, 0.9533678756476683, 1.0, 0.9451073985680191, 0.9671052631578947, 0.9322493224932249, 0.9895833333333334, 0.9295039164490861, 0.9822222222222222, 0.9508196721311475, 0.9583333333333334, 0.989041095890411, 0.9892116182572614]",0.647967,0.622605,0.632769,0.934330
3,0.073000,0.114426,"[0.8636363636363636, 0.9298245614035088, 0.9661016949152542, 0.9558011049723757, 0.9887640449438202, 0.9587628865979382, 1.0, 0.9770867430441899, 1.0, 0.9548022598870056, 1.0, 0.9368421052631579, 1.0, 0.991869918699187, 0.9716312056737588, 0.9904109589041096, 1.0]","[0.8735632183908046, 0.828125, 0.8382352941176471, 0.812206572769953, 0.8934010152284264, 0.9587628865979382, 0.9911111111111112, 0.9255813953488372, 0.9424, 0.9086021505376344, 0.979381443298969, 0.9222797927461139, 0.9563318777292577, 0.953125, 0.9647887323943662, 1.0, 0.9802631578947368]","[0.8685714285714285, 0.8760330578512396, 0.8976377952755905, 0.8781725888324873, 0.9386666666666666, 0.9587628865979382, 0.9955357142857143, 0.9506369426751592, 0.9703459637561779, 0.931129476584022, 0.9895833333333334, 0.9295039164490861, 0.9776785714285714, 0.9721115537848606, 0.9681978798586572, 0.9951823812801102, 0.9900332225913622]",0.610575,0.582524,0.595844,0.943216
4,0.051600,0.114038,"[0.9308176100628931, 0.9827586206896551, 0.9682539682539683, 0.945, 0.9837837837837838, 0.96, 1.0, 0.9660194174757282, 1.0, 0.9705882352941176, 0.9895833333333334, 0.9375, 1.0, 0.9919354838709677, 0.9785714285714285, 0.9917582417582418, 1.0]","[0.8505747126436781, 0.890625, 0.8970588235294118, 0.8873239436619719, 0.9238578680203046, 0.9896907216494846, 0.9911111111111112, 0.9255813953488372, 0.9504, 0.8870967741935484, 0.979381443298969, 0.9326424870466321, 0.9737991266375546, 0.9609375, 0.9647887323943662, 0.9986168741355463, 0.9802631578947368]","[0.8888888888888888, 0.9344262295081968, 0.9312977099236641, 0.9152542372881356, 0.9528795811518325, 0.9746192893401016, 0.9955357142857143, 0.9453681710213777, 0.9745693191140279, 0.9269662921348315, 0.9844559585492227, 0.935064935064935, 0.9867256637168141, 0.9761904761904762, 0.9716312056737588, 0.9951757408683667, 0.9900332225913622]",0.638330,0.614760,0.626119,0.951019
5,0.036900,0.117095,"[0.9235294117647059, 1.0, 0.9682539682539

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/li

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-05-06 14:58:41,145] Trial 24 finished with value: 0.6529140575102025 and parameters: {'learning_rate': 4.845903069261359e-06, 'batch_size': 8, 'epochs': 128, 'attention_dropout': 0.21095300005341866, 'hidden_dropout': 0.09706314663391073, 'weight_decay': 0.02006562614670868}. Best is trial 14 with value: 0.6583227918224134.


Melhores hiperparâmetros na partição de validação: {'learning_rate': 2.8185488646028727e-06, 'batch_size': 8, 'epochs': 64, 'attention_dropout': 0.2963825659492601, 'hidden_dropout': 0.10474093889998445, 'weight_decay': 0.017481229857431214}


In [ ]:
test_results = []

# Hiperparâmetros otimizados na partição de validação
best_hyperparams = {
    'learning_rate': 2.8185488646028727e-06,
    'batch_size': 8,
    'epochs': 64,
    'attention_dropout': 0.2963825659492601,
    'hidden_dropout': 0.10474093889998445,
    'weight_decay': 0.017481229857431214
}

for i in range(1,11):
    if i == 8:
        continue  # Ignorar a partição de validação
    
    test_set = particoes[f"particao_{i}"]
    train_sets = [particoes[f"particao_{j}"] for j in range(1,11) if j not in {i, 8}]
    
    # Concatenando os datasets de treino
    train_set = torch.utils.data.ConcatDataset(train_sets)
    train_set = get_subset(train_set, fraction=1)
    
    # Configuração de checkpoints
    checkpoint_path = os.path.join(checkpoints_dir, f"model_checkpoint_{i}")
    os.makedirs(checkpoint_path, exist_ok=True)  # Garantir que o diretório existe
    print(f"Os checkpoints serão salvos em: {checkpoint_path}")

    # Configuração do treinamento
    final_training_args = TrainingArguments(
        output_dir=checkpoint_path,
        evaluation_strategy="epoch",
        save_strategy="epoch",  # Salvar ao final de cada época
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=best_hyperparams["learning_rate"],
        per_device_train_batch_size=best_hyperparams["batch_size"],
        per_device_eval_batch_size=best_hyperparams["batch_size"],
        num_train_epochs=best_hyperparams["epochs"],  # Garantindo que seja maior que 1
        weight_decay=best_hyperparams["weight_decay"],
        logging_dir=logs_dir,
        logging_steps=10,
        report_to="none",
    ) 
    
    final_model = criar_modelo(num_labels, best_hyperparams["attention_dropout"], best_hyperparams["hidden_dropout"])
    
    final_trainer = Trainer(
        model=final_model,
        args=final_training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics
    )
    
    final_trainer.train()

    # Forçar o salvamento do modelo ao final do treinamento
    final_trainer.save_model(checkpoint_path)

    # Verificar se os arquivos foram salvos corretamente
    print(f"Arquivos salvos em {checkpoint_path}: {os.listdir(checkpoint_path)}")

    test_metrics = final_trainer.evaluate()
    test_results.append(test_metrics)

# Cálculo da média e variância das métricas finais
avg_test_results = {
    metric: np.mean([result[metric] for result in test_results]) for metric in test_results[0]
}
var_test_results = {
    metric: np.var([result[metric] for result in test_results]) for metric in test_results[0]
}

# Criar DataFrame para a tabela desejada

df = pd.DataFrame({
    "aroma_precision": [p["eval_precision_per_class"][0] for p in test_results],
    "aroma_recall": [r["eval_recall_per_class"][0] for r in test_results],
    "aroma_f1": [f["eval_f1_per_class"][0] for f in test_results],

    "consistencia_precision": [p["eval_precision_per_class"][1] for p in test_results],
    "consistencia_recall": [r["eval_recall_per_class"][1] for r in test_results],
    "consistencia_f1": [f["eval_f1_per_class"][1] for f in test_results],

    "cor_precision": [p["eval_precision_per_class"][2] for p in test_results],
    "cor_recall": [r["eval_recall_per_class"][2] for r in test_results],
    "cor_f1": [f["eval_f1_per_class"][2] for f in test_results],

    "sabor_precision": [p["eval_precision_per_class"][3] for p in test_results],
    "sabor_recall": [r["eval_recall_per_class"][3] for r in test_results],
    "sabor_f1": [f["eval_f1_per_class"][3] for f in test_results],

    "classificacao_precision": [p["eval_precision_per_class"][4] for p in test_results],
    "classificacao_recall": [r["eval_recall_per_class"][4] for r in test_results],
    "classificacao_f1": [f["eval_f1_per_class"][4] for f in test_results],

    "equipamento_destilacao_precision": [p["eval_precision_per_class"][5] for p in test_results],
    "equipamento_destilacao_recall": [r["eval_recall_per_class"][5] for r in test_results],
    "equipamento_destilacao_f1": [f["eval_f1_per_class"][5] for f in test_results],

    "graduacao_alcoolica_precision": [p["eval_precision_per_class"][6] for p in test_results],
    "graduacao_alcoolica_recall": [r["eval_recall_per_class"][6] for r in test_results],
    "graduacao_alcoolica_f1": [f["eval_f1_per_class"][6] for f in test_results],

    "nome_bebida_precision": [p["eval_precision_per_class"][7] for p in test_results],
    "nome_bebida_recall": [r["eval_recall_per_class"][7] for r in test_results],
    "nome_bebida_f1": [f["eval_f1_per_class"][7] for f in test_results],

    "nome_local_precision": [p["eval_precision_per_class"][8] for p in test_results],
    "nome_local_recall": [r["eval_recall_per_class"][8] for r in test_results],
    "nome_local_f1": [f["eval_f1_per_class"][8] for f in test_results],

    "nome_organizacao_precision": [p["eval_precision_per_class"][9] for p in test_results],
    "nome_organizacao_recall": [r["eval_recall_per_class"][9] for r in test_results],
    "nome_organizacao_f1": [f["eval_f1_per_class"][9] for f in test_results],

    "nome_pessoa_precision": [p["eval_precision_per_class"][10] for p in test_results],
    "nome_pessoa_recall": [r["eval_recall_per_class"][10] for r in test_results],
    "nome_pessoa_f1": [f["eval_f1_per_class"][10] for f in test_results],

    "preco_precision": [p["eval_precision_per_class"][11] for p in test_results],
    "preco_recall": [r["eval_recall_per_class"][11] for r in test_results],
    "preco_f1": [f["eval_f1_per_class"][11] for f in test_results],

    "recipiente_armazenamento_precision": [p["eval_precision_per_class"][12] for p in test_results],
    "recipiente_armazenamento_recall": [r["eval_recall_per_class"][12] for r in test_results],
    "recipiente_armazenamento_f1": [f["eval_f1_per_class"][12] for f in test_results],

    "tempo_precision": [p["eval_precision_per_class"][13] for p in test_results],
    "tempo_recall": [r["eval_recall_per_class"][13] for r in test_results],
    "tempo_f1": [f["eval_f1_per_class"][13] for f in test_results],

    "tempo_armazenamento_precision": [p["eval_precision_per_class"][14] for p in test_results],
    "tempo_armazenamento_recall": [r["eval_recall_per_class"][14] for r in test_results],
    "tempo_armazenamento_f1": [f["eval_f1_per_class"][14] for f in test_results],

    "tipo_madeira_precision": [p["eval_precision_per_class"][15] for p in test_results],
    "tipo_madeira_recall": [r["eval_recall_per_class"][15] for r in test_results],
    "tipo_madeira_f1": [f["eval_f1_per_class"][15] for f in test_results],

    "volume_precision": [p["eval_precision_per_class"][16] for p in test_results],
    "volume_recall": [r["eval_recall_per_class"][16] for r in test_results],
    "volume_f1": [f["eval_f1_per_class"][16] for f in test_results],

    "precision_macro": [p["eval_precision_macro"] for p in test_results],
    "recall_macro": [r["eval_recall_macro"] for r in test_results],
    "f1_macro": [f["eval_f1_macro"] for f in test_results],
    "accuracy": [a["eval_accuracy"] for a in test_results],
})




df.to_csv("resultados_finais_CACHACA.csv", index=False)

print(df)
print("Média das métricas:", avg_test_results)
print("Variância das métricas:", var_test_results)



df_media_variancia = pd.DataFrame({
    "aroma_precision_mean": [np.mean([r["eval_precision_per_class"][0] for r in test_results])],
    "aroma_precision_var": [np.var([r["eval_precision_per_class"][0] for r in test_results])],
    "aroma_recall_mean": [np.mean([r["eval_recall_per_class"][0] for r in test_results])],
    "aroma_recall_var": [np.var([r["eval_recall_per_class"][0] for r in test_results])],
    "aroma_f1_mean": [np.mean([r["eval_f1_per_class"][0] for r in test_results])],
    "aroma_f1_var": [np.var([r["eval_f1_per_class"][0] for r in test_results])],

    "consistencia_precision_mean": [np.mean([r["eval_precision_per_class"][1] for r in test_results])],
    "consistencia_precision_var": [np.var([r["eval_precision_per_class"][1] for r in test_results])],
    "consistencia_recall_mean": [np.mean([r["eval_recall_per_class"][1] for r in test_results])],
    "consistencia_recall_var": [np.var([r["eval_recall_per_class"][1] for r in test_results])],
    "consistencia_f1_mean": [np.mean([r["eval_f1_per_class"][1] for r in test_results])],
    "consistencia_f1_var": [np.var([r["eval_f1_per_class"][1] for r in test_results])],

    "cor_precision_mean": [np.mean([r["eval_precision_per_class"][2] for r in test_results])],
    "cor_precision_var": [np.var([r["eval_precision_per_class"][2] for r in test_results])],
    "cor_recall_mean": [np.mean([r["eval_recall_per_class"][2] for r in test_results])],
    "cor_recall_var": [np.var([r["eval_recall_per_class"][2] for r in test_results])],
    "cor_f1_mean": [np.mean([r["eval_f1_per_class"][2] for r in test_results])],
    "cor_f1_var": [np.var([r["eval_f1_per_class"][2] for r in test_results])],

    "sabor_precision_mean": [np.mean([r["eval_precision_per_class"][3] for r in test_results])],
    "sabor_precision_var": [np.var([r["eval_precision_per_class"][3] for r in test_results])],
    "sabor_recall_mean": [np.mean([r["eval_recall_per_class"][3] for r in test_results])],
    "sabor_recall_var": [np.var([r["eval_recall_per_class"][3] for r in test_results])],
    "sabor_f1_mean": [np.mean([r["eval_f1_per_class"][3] for r in test_results])],
    "sabor_f1_var": [np.var([r["eval_f1_per_class"][3] for r in test_results])],

    "classificacao_precision_mean": [np.mean([r["eval_precision_per_class"][4] for r in test_results])],
    "classificacao_precision_var": [np.var([r["eval_precision_per_class"][4] for r in test_results])],
    "classificacao_recall_mean": [np.mean([r["eval_recall_per_class"][4] for r in test_results])],
    "classificacao_recall_var": [np.var([r["eval_recall_per_class"][4] for r in test_results])],
    "classificacao_f1_mean": [np.mean([r["eval_f1_per_class"][4] for r in test_results])],
    "classificacao_f1_var": [np.var([r["eval_f1_per_class"][4] for r in test_results])],

    "equipamento_destilacao_precision_mean": [np.mean([r["eval_precision_per_class"][5] for r in test_results])],
    "equipamento_destilacao_precision_var": [np.var([r["eval_precision_per_class"][5] for r in test_results])],
    "equipamento_destilacao_recall_mean": [np.mean([r["eval_recall_per_class"][5] for r in test_results])],
    "equipamento_destilacao_recall_var": [np.var([r["eval_recall_per_class"][5] for r in test_results])],
    "equipamento_destilacao_f1_mean": [np.mean([r["eval_f1_per_class"][5] for r in test_results])],
    "equipamento_destilacao_f1_var": [np.var([r["eval_f1_per_class"][5] for r in test_results])],

    "graduacao_alcoolica_precision_mean": [np.mean([r["eval_precision_per_class"][6] for r in test_results])],
    "graduacao_alcoolica_precision_var": [np.var([r["eval_precision_per_class"][6] for r in test_results])],
    "graduacao_alcoolica_recall_mean": [np.mean([r["eval_recall_per_class"][6] for r in test_results])],
    "graduacao_alcoolica_recall_var": [np.var([r["eval_recall_per_class"][6] for r in test_results])],
    "graduacao_alcoolica_f1_mean": [np.mean([r["eval_f1_per_class"][6] for r in test_results])],
    "graduacao_alcoolica_f1_var": [np.var([r["eval_f1_per_class"][6] for r in test_results])],

    "nome_bebida_precision_mean": [np.mean([r["eval_precision_per_class"][7] for r in test_results])],
    "nome_bebida_precision_var": [np.var([r["eval_precision_per_class"][7] for r in test_results])],
    "nome_bebida_recall_mean": [np.mean([r["eval_recall_per_class"][7] for r in test_results])],
    "nome_bebida_recall_var": [np.var([r["eval_recall_per_class"][7] for r in test_results])],
    "nome_bebida_f1_mean": [np.mean([r["eval_f1_per_class"][7] for r in test_results])],
    "nome_bebida_f1_var": [np.var([r["eval_f1_per_class"][7] for r in test_results])],

    "nome_local_precision_mean": [np.mean([r["eval_precision_per_class"][8] for r in test_results])],
    "nome_local_precision_var": [np.var([r["eval_precision_per_class"][8] for r in test_results])],
    "nome_local_recall_mean": [np.mean([r["eval_recall_per_class"][8] for r in test_results])],
    "nome_local_recall_var": [np.var([r["eval_recall_per_class"][8] for r in test_results])],
    "nome_local_f1_mean": [np.mean([r["eval_f1_per_class"][8] for r in test_results])],
    "nome_local_f1_var": [np.var([r["eval_f1_per_class"][8] for r in test_results])],

    "nome_organizacao_precision_mean": [np.mean([r["eval_precision_per_class"][9] for r in test_results])],
    "nome_organizacao_precision_var": [np.var([r["eval_precision_per_class"][9] for r in test_results])],
    "nome_organizacao_recall_mean": [np.mean([r["eval_recall_per_class"][9] for r in test_results])],
    "nome_organizacao_recall_var": [np.var([r["eval_recall_per_class"][9] for r in test_results])],
    "nome_organizacao_f1_mean": [np.mean([r["eval_f1_per_class"][9] for r in test_results])],
    "nome_organizacao_f1_var": [np.var([r["eval_f1_per_class"][9] for r in test_results])],

    "nome_pessoa_precision_mean": [np.mean([r["eval_precision_per_class"][10] for r in test_results])],
    "nome_pessoa_precision_var": [np.var([r["eval_precision_per_class"][10] for r in test_results])],
    "nome_pessoa_recall_mean": [np.mean([r["eval_recall_per_class"][10] for r in test_results])],
    "nome_pessoa_recall_var": [np.var([r["eval_recall_per_class"][10] for r in test_results])],
    "nome_pessoa_f1_mean": [np.mean([r["eval_f1_per_class"][10] for r in test_results])],
    "nome_pessoa_f1_var": [np.var([r["eval_f1_per_class"][10] for r in test_results])],

    "preco_precision_mean": [np.mean([r["eval_precision_per_class"][11] for r in test_results])],
    "preco_precision_var": [np.var([r["eval_precision_per_class"][11] for r in test_results])],
    "preco_recall_mean": [np.mean([r["eval_recall_per_class"][11] for r in test_results])],
    "preco_recall_var": [np.var([r["eval_recall_per_class"][11] for r in test_results])],
    "preco_f1_mean": [np.mean([r["eval_f1_per_class"][11] for r in test_results])],
    "preco_f1_var": [np.var([r["eval_f1_per_class"][11] for r in test_results])],

    "recipiente_armazanamento_precision_mean": [np.mean([r["eval_precision_per_class"][12] for r in test_results])],
    "recipiente_armazanamento_precision_var": [np.var([r["eval_precision_per_class"][12] for r in test_results])],
    "recipiente_armazanamento_recall_mean": [np.mean([r["eval_recall_per_class"][12] for r in test_results])],
    "recipiente_armazanamento_recall_var": [np.var([r["eval_recall_per_class"][12] for r in test_results])],
    "recipiente_armazanamento_f1_mean": [np.mean([r["eval_f1_per_class"][12] for r in test_results])],
    "recipiente_armazanamento_f1_var": [np.var([r["eval_f1_per_class"][12] for r in test_results])],

    "tempo_precision_mean": [np.mean([r["eval_precision_per_class"][13] for r in test_results])],
    "tempo_precision_var": [np.var([r["eval_precision_per_class"][13] for r in test_results])],
    "tempo_recall_mean": [np.mean([r["eval_recall_per_class"][13] for r in test_results])],
    "tempo_recall_var": [np.var([r["eval_recall_per_class"][13] for r in test_results])],
    "tempo_f1_mean": [np.mean([r["eval_f1_per_class"][13] for r in test_results])],
    "tempo_f1_var": [np.var([r["eval_f1_per_class"][13] for r in test_results])],

    "tempo_armazanamento_precision_mean": [np.mean([r["eval_precision_per_class"][14] for r in test_results])],
    "tempo_armazanamento_precision_var": [np.var([r["eval_precision_per_class"][14] for r in test_results])],
    "tempo_armazanamento_recall_mean": [np.mean([r["eval_recall_per_class"][14] for r in test_results])],
    "tempo_armazanamento_recall_var": [np.var([r["eval_recall_per_class"][14] for r in test_results])],
    "tempo_armazanamento_f1_mean": [np.mean([r["eval_f1_per_class"][14] for r in test_results])],
    "tempo_armazanamento_f1_var": [np.var([r["eval_f1_per_class"][14] for r in test_results])],

    "tipo_madeira_precision_mean": [np.mean([r["eval_precision_per_class"][15] for r in test_results])],
    "tipo_madeira_precision_var": [np.var([r["eval_precision_per_class"][15] for r in test_results])],
    "tipo_madeira_recall_mean": [np.mean([r["eval_recall_per_class"][15] for r in test_results])],
    "tipo_madeira_recall_var": [np.var([r["eval_recall_per_class"][15] for r in test_results])],
    "tipo_madeira_f1_mean": [np.mean([r["eval_f1_per_class"][15] for r in test_results])],
    "tipo_madeira_f1_var": [np.var([r["eval_f1_per_class"][15] for r in test_results])],

    "volume_precision_mean": [np.mean([r["eval_precision_per_class"][16] for r in test_results])],
    "volume_precision_var": [np.var([r["eval_precision_per_class"][16] for r in test_results])],
    "volume_recall_mean": [np.mean([r["eval_recall_per_class"][16] for r in test_results])],
    "volume_recall_var": [np.var([r["eval_recall_per_class"][16] for r in test_results])],
    "volume_f1_mean": [np.mean([r["eval_f1_per_class"][16] for r in test_results])],
    "volume_f1_var": [np.var([r["eval_f1_per_class"][16] for r in test_results])],

    # Cálculo dos valores Macro
    "precision_macro_mean": [np.mean([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(17)
    ])],  # Média de todas as precisões
    "precision_macro_var": [np.var([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(17)
    ])],

    "recall_macro_mean": [np.mean([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(17)
    ])],  # Média de todos os recalls
    "recall_macro_var": [np.var([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(17)
    ])],

    "f1_macro_mean": [np.mean([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(17)
    ])],  # Média de todos os F1
    "f1_macro_var": [np.var([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(17)
    ])],

    # Cálculo de Accuracy
    "accuracy_mean": [np.mean([result["eval_accuracy"] for result in test_results])],
    "accuracy_var": [np.var([result["eval_accuracy"] for result in test_results])]
})



# Exibindo a nova tabela com as médias e variâncias
pd.set_option("display.max_columns", None)  # Mostra todas as colunas
pd.set_option("display.width", None)        # Não quebra linha
print(df_media_variancia)


# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                                      # Fora de qualquer entidade
    'B-CARACTERISTICA_SENSORIAL_AROMA': 1,       # Início da entidade relacionada ao aroma
    'I-CARACTERISTICA_SENSORIAL_AROMA': 2,       # Continuação da entidade relacionada ao aroma
    'B-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': 3,# Início da entidade de consistência
    'I-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': 4,# Continuação da entidade de consistência
    'B-CARACTERISTICA_SENSORIAL_COR': 5,         # Início da entidade de cor
    'I-CARACTERISTICA_SENSORIAL_COR': 6,         # Continuação da entidade de cor
    'B-CARACTERISTICA_SENSORIAL_SABOR': 7,       # Início da entidade de sabor
    'I-CARACTERISTICA_SENSORIAL_SABOR': 8,       # Continuação da entidade de sabor
    'B-CLASSIFICACAO_BEBIDA': 9,                 # Início da classificação da bebida
    'I-CLASSIFICACAO_BEBIDA': 10,                # Continuação da classificação da bebida
    'B-EQUIPAMENTO_DESTILACAO': 11,              # Início da entidade de equipamento de destilação
    'I-EQUIPAMENTO_DESTILACAO': 12,              # Continuação do equipamento de destilação
    'B-GRADUACAO-ALCOOLICA': 13,                 # Início da graduação alcoólica
    'I-GRADUACAO-ALCOOLICA': 14,                 # Continuação da graduação alcoólica
    'B-NOME_BEBIDA': 15,                         # Início do nome da bebida
    'I-NOME_BEBIDA': 16,                         # Continuação do nome da bebida
    'B-NOME_LOCAL': 17,                          # Início do nome de local
    'I-NOME_LOCAL': 18,                          # Continuação do nome de local
    'B-NOME_ORGANIZACAO': 19,                    # Início do nome da organização
    'I-NOME_ORGANIZACAO': 20,                    # Continuação do nome da organização
    'B-NOME_PESSOA': 21,                         # Início do nome de pessoa
    'I-NOME_PESSOA': 22,                         # Continuação do nome de pessoa
    'B-PRECO': 23,                               # Início da entidade de preço
    'I-PRECO': 24,                               # Continuação da entidade de preço
    'B-RECIPIENTE_ARMAZENAMENTO': 25,            # Início da entidade de recipiente de armazenamento
    'I-RECIPIENTE_ARMAZENAMENTO': 26,            # Continuação do recipiente de armazenamento
    'B-TEMPO': 27,                               # Início da entidade de tempo (geral)
    'I-TEMPO': 28,                               # Continuação da entidade de tempo
    'B-TEMPO_ARMAZENAMENTO': 29,                 # Início da entidade de tempo de armazenamento
    'I-TEMPO_ARMAZENAMENTO': 30,                 # Continuação da entidade de tempo de armazenamento
    'B-TIPO_MADEIRA': 31,                        # Início da entidade tipo de madeira
    'I-TIPO_MADEIRA': 32,                        # Continuação da entidade tipo de madeira
    'B-VOLUME': 33,                              # Início da entidade de volume
    'I-VOLUME': 34,                              # Continuação da entidade de volume
}


# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/cachaca_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")




Os checkpoints serão salvos em: ./checkpoints/model_checkpoint_1


/home/juliaribeiro/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.380900,0.324259,"[0.7230769230769231, 0.0, 1.0, 0.3595505617977528, 1.0, 0.0, 0.9907834101382489, 0.88, 0.9804270462633452, 0.8055555555555556, 0.8301886792452831, 0.9148936170212766, 0.8333333333333334, 1.0, 0.9761904761904762, 0.8550247116968699, 0.9965337954939342]","[0.5662650602409639, 0.0, 0.3176470588235294, 0.47058823529411764, 0.07262569832402235, 0.0, 0.9953703703703703, 0.8064516129032258, 0.8746031746031746, 0.14948453608247422, 0.55, 0.9052631578947369, 0.8108108108108109, 0.8211382113821138, 0.984, 0.9737335834896811, 0.9712837837837838]","[0.6351351351351351, 0.0, 0.48214285714285715, 0.40764331210191085, 0.13541666666666666, 0.0, 0.9930715935334873, 0.8416220351951033, 0.924496644295302, 0.25217391304347825, 0.6616541353383458, 0.91005291005291, 0.821917808219178, 0.9017857142857143, 0.9800796812749004, 0.9105263157894737, 0.9837467921300257]",0.571546,0.446490,0.471368,0.755334
2,0.248300,0.185233,"[0.7777777777777778, 0.9583333333333334, 0.9444444444444444, 0.5797101449275363, 0.9776119402985075, 0.979381443298969, 1.0, 0.9505988023952096, 0.9932203389830508, 0.8980891719745223, 0.9230769230769231, 0.9148936170212766, 0.9906103286384976, 1.0, 0.984, 0.9655172413793104, 0.9982638888888888]","[0.6325301204819277, 0.7666666666666667, 0.8, 0.5882352941176471, 0.7318435754189944, 0.9595959595959596, 1.0, 0.9310850439882697, 0.9301587301587302, 0.7268041237113402, 0.75, 0.9052631578947369, 0.9504504504504504, 0.9349593495934959, 0.984, 0.99812382739212, 0.9712837837837838]","[0.6976744186046512, 0.8518518518518519, 0.8662420382165605, 0.583941605839416, 0.8370607028753994, 0.9693877551020408, 1.0, 0.9407407407407408, 0.9606557377049181, 0.8034188034188035, 0.8275862068965517, 0.91005291005291, 0.9701149425287356, 0.9663865546218487, 0.984, 0.981549815498155, 0.9845890410958904]",0.633421,0.582440,0.605410,0.902365
3,0.209400,0.132074,"[0.7692307692307693, 0.9444444444444444, 0.9493670886075949, 0.6851851851851852, 0.9934640522875817, 1.0, 1.0, 0.9727272727272728, 0.9917627677100495, 0.9139784946236559, 0.9305555555555556, 0.9148936170212766, 1.0, 1.0, 0.984, 0.9981132075471698, 1.0]","[0.7831325301204819, 0.85, 0.8823529411764706, 0.5441176470588235, 0.8491620111731844, 0.9696969696969697, 1.0, 0.9413489736070382, 0.9555555555555556, 0.8762886597938144, 0.8375, 0.9052631578947369, 0.954954954954955, 0.9349593495934959, 0.984, 0.9924953095684803, 0.9712837837837838]","[0.7761194029850746, 0.8947368421052632, 0.9146341463414634, 0.6065573770491803, 0.9156626506024096, 0.9846153846153847, 1.0, 0.9567809239940388, 0.973322554567502, 0.8947368421052632, 0.881578947368421, 0.91005291005291, 0.9769585253456221, 0.9663865546218487, 0.984, 0.9952963311382879, 0.9854327335047129]",0.573133,0.544004,0.557745,0.927876
4,0.130700,0.111539,"[0.8581560283687943, 0.8983050847457628, 0.9156626506024096, 0.6956521739130435, 0.9698795180722891, 1.0, 1.0, 0.9831288343558282, 0.9934533551554828, 0.9162303664921466, 0.9444444444444444, 0.9148936170212766, 1.0, 1.0, 0.984, 0.9906716417910447, 1.0]","[0.7289156626506024, 0.8833333333333333, 0.8941176470588236, 0.7058823529411765, 0.8994413407821229, 0.9696969696969697, 1.0, 0.9398826979472141, 0.9634920634920635, 0.9020618556701031, 0.85, 0.9052631578947369, 0.9864864864864865, 0.9349593495934959, 0.984, 0.9962476547842402, 0.9712837837837838]","[0.7882736156351792, 0.8907563025210085, 0.9047619047619048, 0.7007299270072993, 0.9333333333333333, 0.9846153846153847, 1.0, 0.9610194902548725, 0.9782433521353747, 0.9090909090909091, 0.8947368421052632, 0.91005291005291, 0.9931972789115646, 0.9663865546218487, 0.984, 0.9934518241347053, 0.9854327335047129]",0.594981,0.574632,0.584373,0.938080
5,0.077100,0.098807,"[0.80625, 0.9642857142857143, 0.9493670886075949, 0.7623762376237624, 0.9938650306748467, 1.0, 1.0, 0.9805389221556886, 0.9934959349593496, 0.921875,

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

In [8]:


df_media_variancia = pd.DataFrame({
    "data_precision_mean": [np.mean([result["eval_precision_per_class"][0] for result in test_results])],
    "data_precision_var": [np.var([result["eval_precision_per_class"][0] for result in test_results])],
    "data_recall_mean": [np.mean([result["eval_recall_per_class"][0] for result in test_results])],
    "data_recall_var": [np.var([result["eval_recall_per_class"][0] for result in test_results])],
    "data_f1_mean": [np.mean([result["eval_f1_per_class"][0] for result in test_results])],
    "data_f1_var": [np.var([result["eval_f1_per_class"][0] for result in test_results])],

    "evento_precision_mean": [np.mean([result["eval_precision_per_class"][1] for result in test_results])],
    "evento_precision_var": [np.var([result["eval_precision_per_class"][1] for result in test_results])],
    "evento_recall_mean": [np.mean([result["eval_recall_per_class"][1] for result in test_results])],
    "evento_recall_var": [np.var([result["eval_recall_per_class"][1] for result in test_results])],
    "evento_f1_mean": [np.mean([result["eval_f1_per_class"][1] for result in test_results])],
    "evento_f1_var": [np.var([result["eval_f1_per_class"][1] for result in test_results])],

    "fundamento_precision_mean": [np.mean([result["eval_precision_per_class"][2] for result in test_results])],
    "fundamento_precision_var": [np.var([result["eval_precision_per_class"][2] for result in test_results])],
    "fundamento_recall_mean": [np.mean([result["eval_recall_per_class"][2] for result in test_results])],
    "fundamento_recall_var": [np.var([result["eval_recall_per_class"][2] for result in test_results])],
    "fundamento_f1_mean": [np.mean([result["eval_f1_per_class"][2] for result in test_results])],
    "fundamento_f1_var": [np.var([result["eval_f1_per_class"][2] for result in test_results])],

    "local_precision_mean": [np.mean([result["eval_precision_per_class"][3] for result in test_results])],
    "local_precision_var": [np.var([result["eval_precision_per_class"][3] for result in test_results])],
    "local_recall_mean": [np.mean([result["eval_recall_per_class"][3] for result in test_results])],
    "local_recall_var": [np.var([result["eval_recall_per_class"][3] for result in test_results])],
    "local_f1_mean": [np.mean([result["eval_f1_per_class"][3] for result in test_results])],
    "local_f1_var": [np.var([result["eval_f1_per_class"][3] for result in test_results])],

    "organizacao_precision_mean": [np.mean([result["eval_precision_per_class"][4] for result in test_results])],
    "organizacao_precision_var": [np.var([result["eval_precision_per_class"][4] for result in test_results])],
    "organizacao_recall_mean": [np.mean([result["eval_recall_per_class"][4] for result in test_results])],
    "organizacao_recall_var": [np.var([result["eval_recall_per_class"][4] for result in test_results])],
    "organizacao_f1_mean": [np.mean([result["eval_f1_per_class"][4] for result in test_results])],
    "organizacao_f1_var": [np.var([result["eval_f1_per_class"][4] for result in test_results])],

    "pessoa_precision_mean": [np.mean([result["eval_precision_per_class"][5] for result in test_results])],
    "pessoa_precision_var": [np.var([result["eval_precision_per_class"][5] for result in test_results])],
    "pessoa_recall_mean": [np.mean([result["eval_recall_per_class"][5] for result in test_results])],
    "pessoa_recall_var": [np.var([result["eval_recall_per_class"][5] for result in test_results])],
    "pessoa_f1_mean": [np.mean([result["eval_f1_per_class"][5] for result in test_results])],
    "pessoa_f1_var": [np.var([result["eval_f1_per_class"][5] for result in test_results])],

    "produtodelei_precision_mean": [np.mean([result["eval_precision_per_class"][6] for result in test_results])],
    "produtodelei_precision_var": [np.var([result["eval_precision_per_class"][6] for result in test_results])],
    "produtodelei_recall_mean": [np.mean([result["eval_recall_per_class"][6] for result in test_results])],
    "produtodelei_recall_var": [np.var([result["eval_recall_per_class"][6] for result in test_results])],
    "produtodelei_f1_mean": [np.mean([result["eval_f1_per_class"][6] for result in test_results])],
    "produtodelei_f1_var": [np.var([result["eval_f1_per_class"][6] for result in test_results])],

    # Cálculo dos valores Macro
    "precision_macro_mean": [np.mean([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(7)
    ])],  # Média de todas as precisões
    "precision_macro_var": [np.var([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(7)
    ])],

    "recall_macro_mean": [np.mean([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(7)
    ])],  # Média de todos os recalls
    "recall_macro_var": [np.var([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(7)
    ])],

    "f1_macro_mean": [np.mean([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(7)
    ])],  # Média de todos os F1
    "f1_macro_var": [np.var([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(7)
    ])],

    # Cálculo de Accuracy
    "accuracy_mean": [np.mean([result["eval_accuracy"] for result in test_results])],
    "accuracy_var": [np.var([result["eval_accuracy"] for result in test_results])]
})

# Exibindo a nova tabela com as médias e variâncias
print(df_media_variancia)




NameError: name 'df' is not defined

In [1]:
# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                    # Fora de entidade
    'B-LOCAL': 1,               # Início da entidade DATA
    'I-LOCAL': 2,               # Interior da entidade DATA
    'B-ORGANIZACAO': 3,             # Início da entidade EVENTO
    'I-ORGANIZACAO': 4,             # Interior da entidade EVENTO
    'B-PESSOA': 5,        # Início da entidade FUNDapelido
    'I-PESSOA': 6,        # Interior da entidade FUNDapelido
    'B-TEMPO': 7,        # Início da entidade FUNDapelido
    'I-TEMPO': 8, 
}

# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/Ulysses_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")



NameError: name 'final_trainer' is not defined

In [8]:
# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/Ulysses_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")


Arquivo salvo em: /home/juliaribeiro/Ulysses_error_data.txt
